# Prenight Briefing, modified, for {{ params.day_obs }}

This is presently just a playground for building a pre-night briefing.

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2025-05-10"
sim_version = "3.4"

In [ ]:
# Validate the input
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d+\.\d+$', sim_version) is not None

In [ ]:
from IPython.display import display, HTML
import datetime
import sys
import os
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
from astropy.time import Time

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/schedview')

In [ ]:
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.plot

In [ ]:
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
sim_rp = ResourcePath(f'/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db')
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
visits = schedview.collect.read_opsim(
    sim_rp,
    constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}",
    stackers=[
        maf.stackers.TeffStacker(),
        maf.stackers.ObservationStartDatetime64Stacker(),
        maf.stackers.DayObsStacker(),
        maf.stackers.DayObsMJDStacker(),
        maf.stackers.DayObsISOStacker(),
        maf.stackers.OverheadStacker()
    ]        
)
visits = schedview.compute.visits.add_coords_tuple(visits)

### Numbers of exposures, and gaps between them

In [ ]:
overhead_summary = schedview.compute.visits.compute_overhead_summary(visits, night_events.loc['sun_n12_setting','MJD'], night_events.loc['sun_n12_rising','MJD'])
summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
display(HTML(summary_table))

### Map of the visits

In [ ]:
vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
    visits=visits,
    night_date=day_obs_date,
    timezone=timezone,
    observatory=observatory,
)
bokeh.io.show(vmap)

### Altitude plot

In [ ]:
fig = schedview.plot.nightly.plot_alt_vs_time(
    visits=visits, almanac_events=night_events
)
bokeh.io.show(fig)

### Horizon plot

In [ ]:
fig = schedview.plot.nightly.plot_polar_alt_az(visits=visits, legend=True)
bokeh.io.show(fig)

### Table of visits

In [ ]:
displayed_columns = ['start_date', 'fieldRA', 'fieldDec', 'filter', 'visitExposureTime', 'numExposures', 't_eff', 'skyBrightness', 'seeingFwhmEff', 'cloud', 'note']
displayed_visits_df = visits.loc[:, displayed_columns]
with pd.option_context('display.max_rows', 2000):
    display(displayed_visits_df)